# 计算图和动态图机制
## 1. 计算图 Computational graph

计算图是用来<font color=blue>描述运算</font>的有向无环图    
计算图有两个主要元素: 
- 结点(Node): 结点表示数据，如向量，矩阵，张量
- 边(Edge): 边表示运算，如加减乘除卷积等

用计算图表示: $$y = (x+w)*(w+1)$$    
$$a = x + w; b = w + 1; y = a* b$$    

### 1.1 计算图与梯度求道例子:

$$\frac{\partial y}{\partial w}\vert{_{w=1,x=2}} = \frac{\partial y}{\partial a}\frac{\partial a}{\partial w} + \frac{\partial y}{\partial b}\frac{\partial b}{\partial w} $$
$$= b * 1 + a * 1$$
$$= (w + 1) + (x + w)$$
$$= 2*w + x + 1$$
$$= 2*1 + 2 + 1 = 5$$ 

![avatar](picture/梯度求导.png)

In [5]:
import torch

w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)

y.backward()
print(w.grad)

tensor([5.])


---
### 1.2 叶子结点
需要注意的⚠️:
- 叶子结点: 用户创建的节点称为叶子结点，比如x与w，可以通过 is_leaf 来查看张量是否为叶子结点

在每次反向传播 y.backward() 之后，非叶子结点的梯度都会释放，来节约内存

In [7]:
# 查看叶子结点
print("is_leaf:\n", w.is_leaf, x.is_leaf, a.is_leaf, b.is_leaf, y.is_leaf)

# 查看梯度: 非叶子结点的a,b,y都被释放
print("gradient:\n", w.grad, x.grad, a.grad, b.grad, y.grad)

is_leaf:
 True True False False False
gradient:
 tensor([5.]) tensor([2.]) None None None


In [8]:
# 对于非叶子结点，可以通过 retain_grad() 来保存它的梯度不至于被释放
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
a.retain_grad()
b = torch.add(w, 1)
y = torch.mul(a, b)

y.backward()
print("gradient:\n", w.grad, x.grad, a.grad, b.grad, y.grad)

gradient:
 tensor([5.]) tensor([2.]) tensor([2.]) None None


### 1.3 grad_fn
grad_fn: 记录创建该张量时所用的方法 (函数)    
- y.grad_fn = <MulBackward0   
- a.grad_fn = <AddBackward0   
- b.grad_fn = <AddBackward0   

In [11]:
# 查看 grad_fn; x和w是叶子结点，没有梯度函数; a和b是加法得到的; y是乘法得到的
print("grad_fn:\n", w.grad_fn, x.grad_fn, a.grad_fn, b.grad_fn, y.grad_fn)

grad_fn:
 None None <AddBackward0 object at 0x118e30d30> <AddBackward0 object at 0x118e304e0> <MulBackward0 object at 0x118e30ef0>


## 2. 动态图与静态图 (Dynamic vs Static Computational Gaphs)
### 2.1 静态图 TensorFlow
先搭建好框架，数据从最底部输入，数据在流动的过程中计算图的框架不可以改变
![avatar](picture/静态图.png)
### 2.2 动态图 Pytorch
计算图从叶子结点开始建立，计算图根据代码逐层建立好
![avatar](picture/动态图.png)